In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
warnings.filterwarnings('ignore')
import time

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Analytics Vidhya/train.csv')
df.shape

In [ ]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)Q

In [ ]:
df.drop('row_id',axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df.replace([' ','\\N','Null','NaN'],np.nan)
df.isnull().sum()[df.isnull().sum()>0]

In [ ]:
df.fillna(0,inplace=True)

In [ ]:
df[['player_weight','player_height']].describe()

In [ ]:
df[['player_position_1','player_position_2']].nunique()

In [ ]:
df.groupby('competitionId','winner')['rating_num'].mean()

In [ ]:
df[df.columns[pd.Series(df.columns).str.startswith('player_defensive_raw_var_')]]

In [ ]:
df=df[~(df.player_position_1.isin([20.0,22.0,23.0]))]

In [ ]:
df.player_position_1.value_counts().sort_index()

In [ ]:
df.player_position_2.value_counts().sort_index()

In [ ]:
df['position']=np.where(df.player_position_1==df.player_position_2,0,1)

In [ ]:
df[['player_position_1','player_position_2','position']]

In [ ]:
df=pd.get_dummies(df,columns=['winner','team','player_position_1','player_position_2'],drop_first=True)

In [ ]:
df.drop(['team1_system_id','team2_system_id','scout_id'],axis=1,inplace=True)

In [ ]:
x=list(df.columns)
x.remove('rating_num')

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(df[x],df['rating_num'],test_size=.3,random_state=17)
x_train.shape,y_train.shape,x_test.shape,y_test.shape

In [ ]:
total_features=l

In [ ]:
len(total_features)

In [ ]:
len(list(l[1]))

In [ ]:
b=list(l[1])
b.append('rating_num')

In [ ]:
df[b].to_csv('/content/drive/MyDrive/Analytics Vidhya/features.csv')

In [ ]:
feats=list(l[1])

In [ ]:
import lightgbm as lgb

In [ ]:
model=lgb.LGBMRegressor()

model.fit(x_train[feats],y_train)

In [ ]:
x_train['pred']=(model.predict(x_train[feats]))

In [ ]:
r2_score(x_train['pred'],y_train)

In [ ]:
x_test['pred']=(model.predict(x_test[feats]))

In [ ]:
r2_score(x_test['pred'],y_test)

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/Analytics Vidhya/test.csv')

In [ ]:
test.replace([' ','\\N','Null','NaN'],np.nan)
test['position']=np.where(test.player_position_1==test.player_position_2,0,1)
test=pd.get_dummies(test,columns=['winner','team','player_position_1','player_position_2'],drop_first=True)

In [ ]:
test.fillna(0,inplace=True)

In [ ]:
test['rating_num']=(model.predict(test[feats]))

In [ ]:
test[['row_id','rating_num']].to_csv('/content/drive/MyDrive/Analytics Vidhya/prediction2.csv',index=False)

# Only the features selected using Boosted Feature Selector

In [ ]:
df1=pd.read_csv('/content/drive/MyDrive/Analytics Vidhya/features.csv')

In [ ]:
df1.drop('Unnamed: 0',axis=1,inplace=True)
df1.head()

In [ ]:
l=list(df1.columns)
l.remove('rating_num')

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(df1[l],df1['rating_num'],test_size=.3,random_state=15)
x_train.shape,y_train.shape,x_test.shape,y_test.shape

In [ ]:
import lightgbm as lgb
from lightgbm import LGBMRegressor
gbm=lgb.LGBMRegressor()
gbm.fit(x_train,y_train,eval_set=[(x_test,y_test)],eval_metric='r2')

In [ ]:
from sklearn.feature_selection import RFE

rfe = RFE(gbm,n_features_to_select=200,step= 1)
rfe = rfe.fit(x_train, y_train)

# summarize the selection of the attributes
print(rfe.support_)

In [ ]:
params=rfe.get_params()
x_train['pred']=rfe.predict(x_train)

In [ ]:
from sklearn.metrics import r2_score
r2_score(x_train['pred'],y_train)

In [ ]:
x_test['pred']=np.round(rfe.predict(x_test))

In [ ]:
r2_score(x_test['pred'],y_test)

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/Analytics Vidhya/test.csv')

In [ ]:
test.replace([' ','\\N','Null','NaN'],np.nan)
test['position']=np.where(test.player_position_1==test.player_position_2,0,1)
test=pd.get_dummies(test,columns=['winner','team','player_position_1','player_position_2'],drop_first=True)

In [ ]:
test.fillna(0,inplace=True)

In [ ]:
test.isnull().sum()

In [ ]:
test['rating_num']=np.round(rfe.predict(test[l]))

In [ ]:
test[['row_id','rating_num']].to_csv('/content/drive/MyDrive/Analytics Vidhya/predictions3.csv',index=False)

In [ ]:
# summarize the ranking of the attributes
fea_rank_ = pd.DataFrame({'cols':x_train.columns, 'fea_rank':rfe.ranking_})
fea_rank_.loc[fea_rank_.fea_rank > 0].sort_values(by=['fea_rank'], ascending = True)

In [ ]:
fea_rank_.loc[fea_rank_.fea_rank > 0].sort_values(by=['fea_rank'], ascending = True).head(40)['cols']

In [ ]:
sorted(zip(gbm.feature_importances_,x_train.columns),reverse=True)

In [ ]:
feats=['competitionId','team1_other_raw_var_36','player_general_derived_var_156','team1_other_raw_var_46','player_weight','player_height','winner_loser',
       'position','team1_other_raw_var_44','team1_other_raw_var_42','team1_defensive_raw_var_29','team1_other_raw_var_28','team1_defensive_raw_var_24','team1_other_raw_var_20',
       'team1_defensive_raw_var_17','team1_offensive_raw_var_7','team1_offensive_raw_var_1','player_offensive_derived_var_154','player_general_derived_var_148',
       'player_general_derived_var_146','player_defensive_derived_var_138','team1_other_raw_var_49','team1_other_raw_var_52','team1_other_raw_var_57','player_general_var_4',
       'player_other_raw_var_38','player_positional_raw_var_185','player_other_raw_var_197','team2_other_raw_var_49','team1_other_ratio_var_16','player_defensive_derived_var_29',
       'team2_other_ratio_var_25','team1_offensive_derived_var_7','team1_offensive_derived_var_0','team1_defensive_derived_var_17','player_offensive_derived_var_55',
       'team1_other_ratio_var_12','player_defensive_derived_var_8','team2_other_raw_var_93','player_other_raw_var_109',
       'team1_other_ratio_var_14','team1_offensive_ratio_var_9','team1_other_ratio_var_7','team1_other_ratio_var_4','team1_other_ratio_var_1','player_physical_derived_var_133',
       'team1_other_ratio_var_0','team1_other_raw_var_88']

In [ ]:
gbm=lgb.LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=370, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
gbm.fit(x_train[feats],y_train,eval_set=[(x_test[feats],y_test)],eval_metric='r2')

In [ ]:
x_train['pred']=np.round(gbm.predict(x_train[feats]))
r2_score(x_train['pred'],y_train)

In [ ]:
x_test['pred']=np.round(gbm.predict(x_test[feats]))
r2_score(x_test['pred'],y_test)

In [ ]:
test['rating_num']=np.round(gbm.predict(test[feats]))

In [ ]:
test[['row_id','rating_num']].to_csv('/content/drive/MyDrive/Analytics Vidhya/predictions16.csv',index=False)